In [11]:
import pandas as pd
import numpy as np
import joblib
from feast import FeatureStore
from sklearn import metrics

# Your project details
PROJECT_ID = "mlops-iris-22f3002972"
BUCKET_URI = "gs://mlops-iris-output-22f3002972"
MODEL_ARTIFACT_DIR = "iris_classifier/model_feast"

In [13]:
import os

# Create artifacts directory if it doesn't exist
os.makedirs('artifacts', exist_ok=True)

# Download model
print(f"Downloading from: {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/model_feast.joblib")
!gsutil cp {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/model_feast.joblib artifacts/

# Verify download
print("Checking local file...")
!ls -lh artifacts/

# Load model
model = joblib.load('artifacts/model_feast.joblib')
print(f"✓ Model loaded: {type(model).__name__}")

Copying gs://mlops-iris-output-22f3002972/iris_classifier/model_feast/model_feast.joblib...
/ [1 files][  2.5 KiB/  2.5 KiB]                                                
Operation completed over 1 objects/2.5 KiB.                                      
Checking local file...
total 4.0K
-rw-r--r-- 1 jupyter jupyter 2.6K Oct 19 14:29 model_feast.joblib
✓ Model loaded: DecisionTreeClassifier


In [14]:
# Download model
!gsutil cp {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/model_feast.joblib artifacts/

# Load model
model = joblib.load('artifacts/model_feast.joblib')
print(f"✓ Model loaded: {type(model).__name__}")

Copying gs://mlops-iris-output-22f3002972/iris_classifier/model_feast/model_feast.joblib...
/ [1 files][  2.5 KiB/  2.5 KiB]                                                
Operation completed over 1 objects/2.5 KiB.                                      
✓ Model loaded: DecisionTreeClassifier


In [15]:
print("Connecting to Feast Feature Store...")
store = FeatureStore(repo_path="/home/jupyter/feast_repo/iris_features")
print("✓ Connected to Feature Store")

Connecting to Feast Feature Store...
✓ Connected to Feature Store


/opt/conda/lib/python3.10/site-packages/feast/repo_config.py:272: DeprecationWarning: The serialization version below 3 are deprecated. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


In [16]:
print("Fetching features from Feast ONLINE Store...")

# Get features for specific iris samples (fast inference)
entity_rows = [{"iris_id": i} for i in range(1, 21)]

# Fetch from ONLINE store (this is FAST!)
online_features = store.get_online_features(
    features=[
        "iris_features:sepal_length",
        "iris_features:sepal_width",
        "iris_features:petal_length",
        "iris_features:petal_width",
        "iris_features:species",
    ],
    entity_rows=entity_rows,
).to_dict()

# Convert to DataFrame
inference_df = pd.DataFrame({
    'iris_id': online_features['iris_id'],
    'sepal_length': online_features['sepal_length'],
    'sepal_width': online_features['sepal_width'],
    'petal_length': online_features['petal_length'],
    'petal_width': online_features['petal_width'],
    'species': online_features['species'],
})

print(f"✓ Retrieved {len(inference_df)} samples from ONLINE store")
inference_df.head()

Fetching features from Feast ONLINE Store...
✓ Retrieved 20 samples from ONLINE store


/opt/conda/lib/python3.10/site-packages/feast/infra/key_encoding_utils.py:141: UserWarning: Serialization of entity key with version < 3 is removed. Please use version 3 by setting entity_key_serialization_version=3.To reserializa your online store featrues refer -  https://github.com/feast-dev/feast/blob/master/docs/how-to-guides/entity-reserialization-of-from-v2-to-v3.md
  warnings.warn(


,iris_id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [17]:
# Prepare features
X_test = inference_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_true = inference_df['species']

# Predict
predictions = model.predict(X_test)

# Results
results_df = pd.DataFrame({
    'Iris ID': inference_df['iris_id'],
    'Actual': y_true,
    'Predicted': predictions,
    'Match': (y_true == predictions)
})

print("\nPrediction Results:")
print(results_df)

# Accuracy
accuracy = metrics.accuracy_score(y_true, predictions)
print(f"\n✓ Accuracy: {accuracy:.3f}")
print(f"✓ Correct: {(y_true == predictions).sum()}/{len(predictions)}")


Prediction Results:
    Iris ID  Actual Predicted  Match
0         1  setosa    setosa   True
1         2  setosa    setosa   True
2         3  setosa    setosa   True
3         4  setosa    setosa   True
4         5  setosa    setosa   True
5         6  setosa    setosa   True
6         7  setosa    setosa   True
7         8  setosa    setosa   True
8         9  setosa    setosa   True
9        10  setosa    setosa   True
10       11  setosa    setosa   True
11       12  setosa    setosa   True
12       13  setosa    setosa   True
13       14  setosa    setosa   True
14       15  setosa    setosa   True
15       16  setosa    setosa   True
16       17  setosa    setosa   True
17       18  setosa    setosa   True
18       19  setosa    setosa   True
19       20  setosa    setosa   True

✓ Accuracy: 1.000
✓ Correct: 20/20
